In [20]:
# %% [markdown]
# # 💊 6. ADMET & Safety Validation
# **Goal:** Ensure our top drugs are "drug-like" (safe and absorbable).

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, QED



In [21]:
# We load the candidates we generated in the previous steps
try:
    top_candidates = pd.read_csv('../artifacts/top_candidates_for_docking.csv')
    print(f"✅ Loaded {len(top_candidates)} candidates for safety screening.")
except FileNotFoundError:
    print("❌ Error: Could not find 'top_candidates_for_docking.csv'. Did you run Notebook 3?")
    # Create dummy data just so the code doesn't crash if file is missing
    top_candidates = pd.DataFrame({'clean_smiles': ['C']})

✅ Loaded 10 candidates for safety screening.


In [22]:
# Function to calculate Safety Metrics
def check_safety(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return pd.Series({'MW': 0, 'LogP': 0, 'QED_Score': 0, 'Lipinski_Pass': "❌"})

    # 1. Lipinski's Rule of 5 (Oral Absorption)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    
    # 2. QED (Quantitative Estimate of Drug-likeness)
    qed_score = QED.qed(mol)
    
    # 3. Safety Decision
    passed = (mw < 500) and (logp < 5) and (hbd < 5) and (hba < 10)
    
    return pd.Series({
        'MW': mw, 
        'LogP': logp, 
        'QED_Score': qed_score, 
        'Lipinski_Pass': "✅" if passed else "❌"
    })

# Apply to Top Candidates
if not top_candidates.empty:
    safety_results = top_candidates['clean_smiles'].apply(check_safety)
    final_report = pd.concat([top_candidates[['clean_smiles']], safety_results], axis=1)

    print("\n🛡️ FINAL SAFETY REPORT")
    display(final_report.sort_values(by='QED_Score', ascending=False))


🛡️ FINAL SAFETY REPORT


,clean_smiles,MW,LogP,QED_Score,Lipinski_Pass
6,CC1(C)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc...,616.568,3.88700,0.304541,❌
5,CC1(C)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc...,640.556,5.17710,0.274164,❌
4,CC1(C)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cncc...,640.556,5.17710,0.274164,❌
1,CC1(N)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc...,640.556,4.47330,0.260595,❌
9,Cc1ccc(NS(=O)(=O)Nc2ccccc2)c(=O)n1CC(=O)NCc1cc...,474.568,1.58749,0.234833,❌
2,CC1(F)C2CC3OB(C(CCCCN)NC(=O)C4CCCN4C(=O)c4cccc...,643.531,5.48410,0.203266,❌
0,N=C(N)NCSCC(NC(=O)CN(CC1(c2ccccc2)CC1)C(=O)CCc...,511.457,0.85017,0.072341,❌
3,CC(C)(CN(CC(=O)NC(CCNN=C(N)N)B(O)O)C(=O)CCc1cc...,496.421,0.09050,0.070886,❌
7,CC1=CC(C(=O)NC(CCCCN)C(=O)C(=O)NSCc2ccc(C(N)=O...,711.845,2.39100,0.058905,❌
8,CC1=CC(C(=O)NC(CCCCN)C(=O)C(=O)NOCc2ccc(C(N)=O...,695.777,1.67440,0.058655,❌
